In [ ]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [ ]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="deepseek-ai/DeepSeek-V3.2-Exp"),
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ InferenceClientModel - deepseek-ai/DeepSeek-V3.2-Exp ──────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = sql_engine("SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1;")                           
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 1: Duration 3.74 seconds| Input tokens: 2,114 | Output tokens: 80]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Woodrow Wilson

[Step 2: Duration 2.42 seconds| Input tokens: 4,390 | Output tokens: 123]

'Woodrow Wilson'

In [ ]:
table_name = "waiters"
waiters = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
insert_rows_into_table(rows, waiters)

In [ ]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


In [ ]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct"),
)

agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Perform the SQL query                                                                                          
  query = """                                                                                                      
  SELECT w.waiter_name, SUM(r.tip) as total_tips                                                                   
  FROM waiters w                                                                                                   
  JOIN receipts r ON w.receipt_id = r.receipt_id                                                                   
  GROUP BY w.waiter_name                                                                                           
  """                                                                                                              
  result = sql_engine(query=query)                                                                                 
                                                                                                                   
  # Convert the result to a DataFrame                                                                              
  df = pd.read_csv(pd.compat.StringIO(result))                                                                     
  print(df)                                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Warning to user: Code execution failed due to an unauthorized import - Consider passing said import under 
`additional_authorized_imports` when initializing your CodeAgent.

Code execution failed at line 'import pandas as pd' due to: InterpreterError: Import of pandas is not allowed. 
Authorized imports are: ['unicodedata', 're', 'random', 'datetime', 'statistics', 'math', 'stat', 'itertools', 
'collections', 'time', 'queue'\]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Perform the SQL query                                                                                          
  query = """                                                                                                      
  SELECT w.waiter_name, SUM(r.tip) as total_tips                                                                   
  FROM waiters w                                                                                                   
  JOIN receipts r ON w.receipt_id = r.receipt_id                                                                   
  GROUP BY w.waiter_name                                                                                           
  ORDER BY total_tips DESC                                                                                         
  LIMIT 1                                                                                                          
  """                                                                                                              
  result = sql_engine(query=query)                                                                                 
                                                                                                                   
  # Debugging the result                                                                                           
  print("SQL Query Result:")                                                                                       
  print(result)                                                                                                    
                                                                                                                   
  # Strip the result and extract the waiter's name                                                                 
  result_parts = result.strip().split(',')                                                                         
  waiter_name = result_parts[0].replace("('", "").replace("',", "").strip()                                        
                                                                                                                   
  print(f"The waiter with the most total tips is {waiter_name}.")                                                  
  final_answer(waiter_name)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
SQL Query Result:

('Michael Watts', 5.67)
The waiter with the most total tips is Michael Watts'.

Final answer: Michael Watts'

[Step 7: Duration 10.06 seconds| Input tokens: 26,510 | Output tokens: 1,659]

"Michael Watts'"